In [ ]:
!pip install transformers
!pip install pytorch-transformers
import pandas as pd
import torch
import numpy as np
from tqdm import tqdm
from transformers import RobertaModel, RobertaTokenizer
from transformers import RobertaForSequenceClassification, RobertaConfig


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.4/176.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 85.6 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.69
    Uninstalling botocore-1.34.69:
      Successfully uninstalled botocore-1.34.69
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.29.165 which is incompatible.


In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/Anuj_Depression_detection

/content/gdrive/MyDrive/Anuj_Depression_detection


## Loading  data

In [5]:
source_url = "sentiment_tweets3.csv"
df = pd.read_csv(source_url)


In [6]:
df.head()

,Index,message to examine,label (depression result)
0,106,just had a real good moment. i missssssssss hi...,0
1,217,is reading manga http://plurk.com/p/mzp1e,0
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0
3,288,@lapcat Need to send 'em to my accountant tomo...,0
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0


In [ ]:
df.groupby(["label (depression result)"]).size()

label (depression result)
0    8000
1    2314
dtype: int64

## Data Preprocessing

In [ ]:
#from transformers import RobertaTokenizer

tokenizer =  RobertaTokenizer.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:

sample = 'Hey my name is RoBERTa'
# truncation : if it is True then we allow bert to truncated every sequence it's length is higher then max_length
# return_tensors : the type of tensors that will be returned (as we are using pytorch then we set "pt")
bert_input  = tokenizer(sample,padding="max_length",max_length=15,truncation=True,return_tensors="pt")



In [ ]:
print(bert_input["input_ids"])
# input_ids are the id representation of each token
# we can decode these inputs to get the original sequence
print(tokenizer.decode(bert_input["input_ids"][0] ))
# the code 102 is for the [SEP] token and the 0 is for [PAD] token

tensor([[    0, 13368,   127,   766,    16,  3830, 11126, 38495,     2,     1,
             1,     1,     1,     1,     1]])
<s>Hey my name is RoBERTa</s><pad><pad><pad><pad><pad><pad>


In [ ]:
print(bert_input["attention_mask"])
# the attention_mask identified whether the token is a real word or just a token padding
# it's 1 for the real words, the CLS and the SEP tokens, and for the pad token is 0


tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10314 entries, 0 to 10313
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Index                      10314 non-null  int64 
 1   message to examine         10314 non-null  object
 2   label (depression result)  10314 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 241.9+ KB


### Dataset Class

Now that we know which are the ouputs of our Roberta tokenizer we are going to build a Dataset Class for our depression Dataset

In [ ]:
from transformers import RobertaTokenizer
import torch
import numpy as np

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

class Dataset(torch.utils.data.Dataset):
    def __init__(self, df):
        # Extract labels from the DataFrame
        self.labels = df["label"].values

        # Tokenize texts using RobertaTokenizer
        self.texts = [tokenizer.encode_plus(text, padding='max_length', max_length=512, truncation=True, return_tensors="pt") for text in df["text"]]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]



Now after creating the Dataset Class let's split our dataset into train,validation and test sets

* training set contains : 80%
* test and validation contains : 10% each

In [ ]:
df_train, df_valid,df_test = np.split(df.sample(frac=1,random_state=42),[int(.8*len(df)), int(.9*len(df))])



/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8251 entries, 2458 to 8488
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Index                      8251 non-null   int64 
 1   message to examine         8251 non-null   object
 2   label (depression result)  8251 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 257.8+ KB


In [ ]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1031 entries, 5607 to 6783
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Index                      1031 non-null   int64 
 1   message to examine         1031 non-null   object
 2   label (depression result)  1031 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 32.2+ KB


In [ ]:
class RoBertaClassifier(torch.nn.Module):
    def __init__(self,dropout=0.5):
        super(RoBertaClassifier,self).__init__()
        self.bert=RobertaModel.from_pretrained("roberta-base")
        self.dropout = torch.nn.Dropout(dropout)
        # bert output a vector of size 768
        self.lin = torch.nn.Linear(768,5)
        self.relu = torch.nn.ReLU()
    def forward(self,input_id,mask, return_dict):
        # as output, the bert model give us first the embedding vector of all the tokens of the sequence
        # second we get the embedding vector of the CLS token.
        # fot a classification task it's enough to use this embedding for our classifier
        _,pooled_output = self.bert(input_ids= input_id,attention_mask = mask,return_dict = return_dict)
        dropout_output = self.dropout(pooled_output)
        linear_output  = self.lin(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer


## Training  

In [ ]:
# we are creating a standard pytorch training loop
from torch.utils.data import Dataset, DataLoader
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.labels = df["label (depression result)"].values
        self.texts = [tokenizer.encode_plus(text, padding='max_length', max_length=512, truncation=True, return_tensors="pt") for text in df["message to examine"]]

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]
def train(model, train_data, val_data, learning_rate, epochs=5):
    #creating a custom Dataset objects using the training and validation data
    train, val = CustomDataset(train_data), CustomDataset(val_data)
    #creating dataloaders
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):
                #print(f"the train input : {train_input}")
                #print(f"train label : {train_label}")

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
    #             print(input_id.shape)

                # get the predictions
                output = model(input_id, mask,return_dict=False)

                #the output is a vector of 5 values (categs)
    #             print(output)
    #             print("the output shape is" ,  output.shape)
    #             print(train_label)

                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()

                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc
                # updating the Gradient Descent and Backpropagation operation
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            # now we evaluate on the validation data
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask, return_dict=False)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')



In [ ]:

EPOCHS = 5
model = RoBertaClassifier()
learning_rate = 1e-5


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train(model, df_train, df_valid, learning_rate, EPOCHS)

100%|██████████| 4126/4126 [08:51<00:00,  7.76it/s]


Epochs: 1 | Train Loss:  0.014                 | Train Accuracy:  0.993                 | Val Loss:  0.008                 | Val Accuracy:  0.995


100%|██████████| 4126/4126 [08:49<00:00,  7.79it/s]


Epochs: 2 | Train Loss:  0.002                 | Train Accuracy:  0.999                 | Val Loss:  0.005                 | Val Accuracy:  0.999


100%|██████████| 4126/4126 [08:49<00:00,  7.80it/s]


Epochs: 3 | Train Loss:  0.002                 | Train Accuracy:  0.999                 | Val Loss:  0.005                 | Val Accuracy:  0.999


100%|██████████| 4126/4126 [08:49<00:00,  7.79it/s]


Epochs: 4 | Train Loss:  0.001                 | Train Accuracy:  1.000                 | Val Loss:  0.005                 | Val Accuracy:  0.999


100%|██████████| 4126/4126 [08:49<00:00,  7.79it/s]


Epochs: 5 | Train Loss:  0.001                 | Train Accuracy:  1.000                 | Val Loss:  0.005                 | Val Accuracy:  0.999


In [ ]:
print(model)

RoBertaClassifier(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Laye